In [31]:
import os
import re
import ast
import torch
import json
import huggingface_hub

from tqdm import tqdm
from openai import OpenAI

## LLM 정의 및 프롬프트 hub
from langchain import hub
from langchain_openai import ChatOpenAI

## Function call
from langchain.tools.retriever import create_retriever_tool

## jsonl 파일 로더
from langchain.document_loaders import JSONLoader

## 인코더, 벡터 DB
from langchain.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

## Huggingface와 Langchain 연동
from langchain_huggingface.llms import HuggingFacePipeline
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

from langchain.tools import Tool
from langchain.agents import initialize_agent, AgentType


from transformers import AutoModelForCausalLM, AutoTokenizer

from dotenv import load_dotenv
load_dotenv("../keys.env")

openai_api_key = os.getenv('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = openai_api_key

hf_token = os.getenv("HF_TOKEN")
huggingface_hub.login(hf_token)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/pervinco/.cache/huggingface/token
Login successful


## documents.jsonl 로드 및 처리

1. 문서 데이터를 encoder 모델로 임베딩.
2. Faiss를 벡터 DB로 정하고 벡터를 저장.

In [2]:
loader = JSONLoader(
    file_path='../dataset/documents.jsonl', 
    json_lines=True, 
    jq_schema='.content'  # 'content' 필드만 추출
)
documents = loader.load()
print(len(documents))

print(documents[0])

4272
page_content='건강한 사람이 에너지 균형을 평형 상태로 유지하는 것은 중요합니다. 에너지 균형은 에너지 섭취와 에너지 소비의 수학적 동등성을 의미합니다. 일반적으로 건강한 사람은 1-2주의 기간 동안 에너지 균형을 달성합니다. 이 기간 동안에는 올바른 식단과 적절한 운동을 통해 에너지 섭취와 에너지 소비를 조절해야 합니다. 식단은 영양가 있는 식품을 포함하고, 적절한 칼로리를 섭취해야 합니다. 또한, 운동은 에너지 소비를 촉진시키고 근육을 강화시킵니다. 이렇게 에너지 균형을 유지하면 건강을 유지하고 비만이나 영양 실조와 같은 문제를 예방할 수 있습니다. 따라서 건강한 사람은 에너지 균형을 평형 상태로 유지하는 것이 중요하며, 이를 위해 1-2주의 기간 동안 식단과 운동을 조절해야 합니다.' metadata={'source': '/home/pervinco/Upstage_Ai_Lab/Final/IR/dataset/documents.jsonl', 'seq_num': 1}


In [3]:
## OpenAIEmbeddings로 임베딩
# embedding_model = OpenAIEmbeddings()
# embeddings = embedding_model.embed_documents([doc.page_content for doc in documents])

## 벡터를 FAISS에 저장
# faiss_index = FAISS.from_texts([doc.page_content for doc in documents], embedding_model)

## Hugging Face Embeddings 설정
model_name = "jhgan/ko-sroberta-multitask"
model_kwargs = {"device": "cuda:0"}
encode_kwargs = {"normalize_embeddings": False}

embedding_model = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs)

## 문서 내용 가져오기
contents = [doc.page_content for doc in documents]

## 벡터 DB 생성 및 저장
vector_db = FAISS.from_texts(texts=contents, embedding=embedding_model)

## 검색기 설정
retriever = vector_db.as_retriever(search_kwargs={"k": 10})

## 벡터 DB에 저장된 벡터의 수 확인
print(f"벡터 DB에 저장된 벡터의 수: {vector_db.index.ntotal}")
print(f"입력한 텍스트의 수: {len(contents)}")

/home/pervinco/miniconda3/envs/IR/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


벡터 DB에 저장된 벡터의 수: 4272
입력한 텍스트의 수: 4272


In [4]:
retriever.invoke("에너지 균형을 유지하는 방법은 무엇인가요?")

[Document(metadata={}, page_content='건강한 사람이 에너지 균형을 평형 상태로 유지하는 것은 중요합니다. 에너지 균형은 에너지 섭취와 에너지 소비의 수학적 동등성을 의미합니다. 일반적으로 건강한 사람은 1-2주의 기간 동안 에너지 균형을 달성합니다. 이 기간 동안에는 올바른 식단과 적절한 운동을 통해 에너지 섭취와 에너지 소비를 조절해야 합니다. 식단은 영양가 있는 식품을 포함하고, 적절한 칼로리를 섭취해야 합니다. 또한, 운동은 에너지 소비를 촉진시키고 근육을 강화시킵니다. 이렇게 에너지 균형을 유지하면 건강을 유지하고 비만이나 영양 실조와 같은 문제를 예방할 수 있습니다. 따라서 건강한 사람은 에너지 균형을 평형 상태로 유지하는 것이 중요하며, 이를 위해 1-2주의 기간 동안 식단과 운동을 조절해야 합니다.'),
 Document(metadata={}, page_content="에너지 전달은 다양한 형태와 방식으로 이루어집니다. 에너지는 한 형태에서 다른 형태로 전달될 수 있으며, 이는 우리 일상 생활에서도 많이 경험할 수 있습니다. 예를 들어, 태양에서 나오는 에너지는 태양광 전지를 통해 전기 에너지로 변환될 수 있습니다. 또한, 운동 에너지는 자전거의 페달을 밟으면서 전기 에너지로 변환될 수 있습니다. 이처럼 에너지 전달은 다양한 종류와 방식을 가지고 있습니다.\n\n하지만, 모든 종류의 에너지 전달을 가장 알맞게 설명하는 명제는 '사용할 수 있는 에너지의 감소를 초래합니다.'입니다. 에너지는 전달되는 과정에서 일부가 손실되기 때문에, 전달된 에너지의 양은 원래의 양보다 적어집니다. 이러한 손실은 에너지의 효율성을 나타내는 중요한 요소 중 하나입니다. 에너지 전달 과정에서 발생하는 손실을 최소화하기 위해 우리는 다양한 기술과 방법을 개발하고 연구하고 있습니다.\n\n에너지 전달은 우리의 삶과 사회에 매우 중요한 역할을 합니다. 우리는 에너지를 사용하여 가정이나 사무실을 데워주고, 차량을 움직이게 하며, 전자기기를 작동시킵니다

In [5]:
retriever.invoke("파이썬")

[Document(metadata={}, page_content='파이썬 3에서 "a" + "ab" 문장의 출력은 "aab"입니다. 파이썬은 문자열을 연결하기 위해 + 연산자를 사용할 수 있습니다. 따라서 "a"와 "ab"를 연결하면 "aab"가 됩니다. 이는 파이썬의 문자열 연산의 한 예시입니다. 파이썬은 문자열을 다루는데 매우 편리한 기능을 제공하며, 문자열을 조작하고 처리하는 다양한 방법을 제공합니다. 이를 통해 파이썬을 사용하여 다양한 문자열 작업을 수행할 수 있습니다.'),
 Document(metadata={}, page_content='파이썬에서 4*1**4의 출력은 4입니다. 파이썬은 강력한 프로그래밍 언어로, 수학적인 연산도 쉽게 처리할 수 있습니다. 이 연산은 4를 1에 4번 곱한 결과를 나타냅니다. 파이썬에서 ** 연산자는 거듭제곱을 의미하며, 1의 4제곱은 1을 4번 곱한 것과 같습니다. 따라서 4*1**4는 4를 의미합니다. 파이썬은 다양한 연산을 지원하며, 이를 통해 다양한 계산을 수행할 수 있습니다.'),
 Document(metadata={}, page_content='파이썬 3에서 "abc"[::-1]의 출력은 \'cba\'입니다. 파이썬에서는 문자열을 뒤집을 때 슬라이싱을 사용할 수 있습니다. 슬라이싱은 시작 인덱스, 끝 인덱스, 그리고 스텝 값을 지정하여 원하는 부분을 추출하는 기능입니다. \'abc\'[::-1]은 문자열 \'abc\'를 뒤에서부터 한 글자씩 추출하여 새로운 문자열 \'cba\'를 생성합니다. 이렇게 생성된 문자열이 출력됩니다. 파이썬에서는 문자열을 다양한 방법으로 조작할 수 있으며, 슬라이싱은 그 중 하나의 방법입니다.'),
 Document(metadata={}, page_content='파이썬에서 4 + 3 % 2는 5의 값을 가집니다. 이는 파이썬의 연산자 우선순위에 따라 계산되며, 먼저 3 % 2를 계산하면 1이 나옵니다. 그리고 4와 1을 더하면 최종적으로 5가 됩니다. 파이썬은 산술 연산을 할 때

## eval.jsonl 로드 및 처리

```eval.jsonl``` 파일 로드

In [6]:
def load_jsonl(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return [json.loads(line) for line in f]

eval_data = load_jsonl("../dataset/eval.jsonl")
print(len(eval_data))
print(eval_data[0].keys())

220
dict_keys(['eval_id', 'msg'])


In [7]:
def convert_to_dialogue_format(messages):
    dialogue = ""
    for msg in messages:
        role = msg['role']
        content = msg['content']
        if role == 'user':
            dialogue += f"user: {content}\n"
        elif role == 'assistant':
            dialogue += f"assistant: {content}\n"
    return dialogue

In [8]:
sample1 = eval_data[0]['msg']
print(sample1)

dialogue1 = convert_to_dialogue_format(sample1)
print(dialogue1)

[{'role': 'user', 'content': '나무의 분류에 대해 조사해 보기 위한 방법은?'}]
user: 나무의 분류에 대해 조사해 보기 위한 방법은?



In [9]:
sample2 = eval_data[8]['msg']
print(sample2)

dialogue2 = convert_to_dialogue_format(sample2)
print(dialogue2)

[{'role': 'user', 'content': '이란 콘트라 사건이 뭐야'}, {'role': 'assistant', 'content': '이란-콘트라 사건은 로널드 레이건 집권기인 1986년에 레이건 행정부와 CIA가 적성국이었던 이란에게 무기를 몰래 수출한 대금으로 니카라과의 우익 성향 반군 콘트라를 지원하면서 동시에 반군으로부터 마약을 사들인 후 미국에 판매하다가 발각되어 큰 파장을 일으킨 사건입니다.'}, {'role': 'user', 'content': '이 사건이 미국 정치에 미친 영향은?'}]
user: 이란 콘트라 사건이 뭐야
assistant: 이란-콘트라 사건은 로널드 레이건 집권기인 1986년에 레이건 행정부와 CIA가 적성국이었던 이란에게 무기를 몰래 수출한 대금으로 니카라과의 우익 성향 반군 콘트라를 지원하면서 동시에 반군으로부터 마약을 사들인 후 미국에 판매하다가 발각되어 큰 파장을 일으킨 사건입니다.
user: 이 사건이 미국 정치에 미친 영향은?



In [10]:
sample3 = eval_data[21]['msg']
print(sample3)

dialogue3 = convert_to_dialogue_format(sample3)
print(dialogue3)

[{'role': 'user', 'content': '요새 너무 힘들다.'}]
user: 요새 너무 힘들다.



## LLM agent

1. LLama3 모델, 토크나이저 로드
2. Fucntion Call 정의

상황별로 내가 정해주는게 아니라 LLM이 알아서, 스스로 판단하는 것이 핵심.

다음과 같은 처리 파이프라인을 정의하고자 함.
1. 질의가 멀티턴(multi-turn) 대화인 경우 LLM이 정리해서 Standalone Query를 생성하고 멀티턴이 아닌 경우 건너뛴다.
2. 질의가 ```과학상식```에 해당한다면 encoder로 전달해 임베딩하고 벡터 db로 전달해 유사도 기반 검색을 수행한다. 그렇지 않은 경우 "답변할 수 없습니다."로 처리한다.

Reference

- [https://python.langchain.com/docs/integrations/chat/](https://python.langchain.com/docs/integrations/chat/)
- [https://github.com/sionic-ai/xionic-ko-llama-3-70b](https://github.com/sionic-ai/xionic-ko-llama-3-70b)

In [9]:
llm = ChatOpenAI(base_url="http://sionic.chat:8001/v1",
                 api_key="934c4bbc-c384-4bea-af82-1450d7f8128d",
                 model="xionic-ko-llama-3-70b",
                 temperature=0.2)

In [ ]:
prompt = hub.pull("hwchase17/react-chat-json")

In [11]:
def create_standalone_query(query):
    if isinstance(query, str) and "user:" in query and "assistant:" in query:
        conversation_lines = query.split("\n")
        
        # 대화의 전체 맥락을 활용하여 standalone query 생성
        standalone_query = " ".join([line.strip() for line in conversation_lines if line])

        return {
            "query": standalone_query,
            "multi-turn": True
        }
    else:
        return {
            "query": query.strip(),
            "multi-turn": False
        }


In [ ]:
# Tool definition
standalone_query_tool = Tool(
    name="Standalone Query Generator",
    func=create_standalone_query,
    description="This tool processes dialogue history and generates a standalone query. \
                 For single-turn dialogues, it returns 'multi-turn: False' and the query unchanged as 'query: query'. \
                 For multi-turn dialogues, the tool extracts the user's last question and generates a concise standalone query based on the context of previous interactions. \
                 It returns 'query: standalone_query' and 'multi-turn: True'. \
                 The purpose is to enable the LLM to efficiently summarize dialogue history and generate actionable queries for further processing."
)



# 에이전트 초기화
tools = [standalone_query_tool]
agent = initialize_agent(
    tools=tools,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    llm=ChatOpenAI(),
    verbose=True
)

In [ ]:
output_single_turn = agent.invoke(dialogue1)
print(output_single_turn)

In [ ]:
output_multi_turn = agent.invoke(dialogue2)
print(output_multi_turn)

In [41]:
client = OpenAI()

In [42]:
def clean_json_response(response):
    # 코드 블록(예: ```json, ```) 제거
    cleaned_response = re.sub(r'```(?:json)?', '', response).strip()
    return cleaned_response

In [43]:
standalone_content = ("입력된 내용이 한 줄의 문장인지 여러 줄의 대화 내용인지 분류하세요. 반드시 JSON 형식으로 응답하며, 키와 값은 모두 이중 따옴표로 감쌉니다."
                      "- 단일 문장인 경우 : {\"multi_turn\": false, \"query\": \"입력 문장\"} "
                      "- 여러 줄의 대화 내용인 경우 : {\"multi_turn\": true, \"query\": \"대화 내용을 종합하여 만든 새로운 질문\"}")


def create_standalone_query(query):
    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": standalone_content},
            {"role": "user", "content": query}
        ],
    )
    
    response = completion.choices[0].message.content
    cleaned_response = clean_json_response(response)
    
    try:
        json_response = json.loads(cleaned_response)
    except json.JSONDecodeError:
        return {"error": "Invalid JSON response", "response": response}
    
    return json_response

In [51]:
domain_content = (
    "입력된 텍스트가 상식에 대한 질문인지 아니면 단순한 일상적인 대화인지 구분하세요."
    " - 상식에 대한 질문이란, 사용자가 지식이나 정보를 얻기 위해 하는 질문입니다. 예: '나무의 분류에 대해 조사하는 방법은?', 'Dmitri Ivanovsky가 누구야?', '남녀 관계에서 정서적인 행동이 왜 중요해?'"
    " - 일상적인 대화란, 주로 감정이나 의견을 표현하거나 대화의 흐름을 유지하기 위한 내용입니다. 예: '요새 너무 힘들다.', '니가 대답을 잘해줘서 너무 신나!', '이제 그만 얘기해!', '오늘 너무 즐거웠다!', ''너는 누구니?', '너는 어떤 능력을 가지고 있니?"
    " 반드시 JSON 형식으로 응답하세요. 키와 값은 모두 이중 따옴표로 감싸야 합니다."
    " - 상식에 대한 질문인 경우: {\"out_of_domain\": false, \"query\": \"입력된 쿼리를 그대로 반환\"}"
    " - 일상적인 대화인 경우: {\"out_of_domain\": true, \"query\": \"적절한 응답을 할 수 없습니다.\"}"
)


def domain_check(query):
    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": domain_content},
            {"role": "user", "content": query}
        ],
    )
    
    response = completion.choices[0].message.content
    cleaned_response = clean_json_response(response)
    
    try:
        json_response = json.loads(cleaned_response)
    except json.JSONDecodeError:
        return {"error": "Invalid JSON response", "response": response}
    
    return json_response

In [47]:
result1 = create_standalone_query(dialogue1)
print(result1)

print(domain_check(result1['query']))

{'multi_turn': False, 'query': '나무의 분류에 대해 조사해 보기 위한 방법은?'}
{'out_of_domain': False, 'query': '나무의 분류에 대해 조사해 보기 위한 방법은?'}


In [48]:
result2 = create_standalone_query(dialogue2)
print(result2)

print(domain_check(result2['query']))

{'multi_turn': True, 'query': '이란-콘트라 사건이 미국 정치에 미친 영향은 무엇인가?'}
{'out_of_domain': False, 'query': '이란-콘트라 사건이 미국 정치에 미친 영향은 무엇인가?'}


In [49]:
result3 = create_standalone_query(dialogue3)
print(result3)

print(domain_check(result3['query']))

{'multi_turn': False, 'query': '요새 너무 힘들다.'}
{'out_of_domain': True, 'query': '적절한 응답을 할 수 없습니다.'}


In [56]:
arr = []
for data in eval_data:
    dialogue = convert_to_dialogue_format(data['msg'])

    interm_query = create_standalone_query(dialogue)    
    final_query = domain_check(interm_query['query'])
    print(f"eval_id : {data['eval_id']}\norg : {dialogue}\nstd_query : {interm_query}\nfinal_query : {final_query}\n")
    
    if final_query['out_of_domain']:
        print("*" * 20)
        print(f"response : {final_query['query']}")
        print("*" * 20)
        arr.append({'eval_id' : data['eval_id'], 'msg' : dialogue, 'final_query' : final_query})

    print()

eval_id : 78
org : user: 나무의 분류에 대해 조사해 보기 위한 방법은?

std_query : {'multi_turn': False, 'query': '나무의 분류에 대해 조사해 보기 위한 방법은?'}
final_query : {'out_of_domain': False, 'query': '나무의 분류에 대해 조사해 보기 위한 방법은?'}


eval_id : 213
org : user: 각 나라에서의 공교육 지출 현황에 대해 알려줘.

std_query : {'multi_turn': False, 'query': '각 나라에서의 공교육 지출 현황에 대해 알려줘.'}
final_query : {'out_of_domain': False, 'query': '각 나라에서의 공교육 지출 현황에 대해 알려줘.'}


eval_id : 107
org : user: 기억 상실증 걸리면 너무 무섭겠다.
assistant: 네 맞습니다.
user: 어떤 원인 때문에 발생하는지 궁금해.

std_query : {'multi_turn': True, 'query': '기억 상실증이 발생하는 원인은 무엇인가요?'}
final_query : {'out_of_domain': False, 'query': '기억 상실증이 발생하는 원인은 무엇인가요?'}


eval_id : 81
org : user: 통학 버스의 가치에 대해 말해줘.

std_query : {'multi_turn': False, 'query': '통학 버스의 가치에 대해 말해줘.'}
final_query : {'out_of_domain': False, 'query': '통학 버스의 가치에 대해 말해줘.'}


eval_id : 280
org : user: Dmitri Ivanovsky가 누구야?

std_query : {'multi_turn': False, 'query': 'Dmitri Ivanovsky가 누구야?'}
final_query : {'out_of_domain': False, 'query': 'D

In [57]:
print(len(arr))

for a in arr:
    print(a['eval_id'])

20
276
261
283
32
94
90
220
245
229
247
67
57
2
227
301
222
83
64
103
218
